In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
pd.set_option('max_colwidth', 400)

In [3]:
# API key and URL
api_key = 'Bearer QjMXje-s1yL6FsIL_x1SynQ65ZyMeWyMWHlxJ8iiPdUylOikFh5ZPwrznAmo4rh33LJXYB4qUqXeRBDg1zyULiT3XV0YFoC-HEfn6zRLiIVfvJ93wF79OrOoNON3ZXYx'
api_url = 'https://api.yelp.com/v3/businesses/search'

# Params
params = {
    'term': 'school',
    'location': 'Austin',
    'categories': 'schools',
    'limit': 20,
    'offset': 0
}

# Set up headers
headers = {
    'Authorization': api_key
}

# Empty list for results
all_schools = []
results_per_request = 20

while True:
    # API request
    response = requests.get(api_url, params=params, headers=headers)

    # If request successful
    if response.status_code == 200:
        data = response.json()
        businesses = data.get('businesses', [])
        all_schools.extend(businesses)

        # Check if more results
        if len(businesses) < results_per_request:
            break

        # Update offset for the next request
        params['offset'] += results_per_request

    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        break

# Create list of dictionaries
school_data = []
for business in all_schools:
    coordinates = business.get('coordinates', {})
    location=business.get('location',{})
    school_data.append({
        'Name': business.get('name', ''),
        'Address': ', '.join(business.get('location', {}).get('display_address', [])),
        'Ratings': business.get('rating', 0),
        'Latitude': coordinates.get('latitude', 0),
        'Longitude': coordinates.get('longitude', 0),
        'zipcode':  location.get('zip_code',0)
    })

# Create a Pandas DataFrame
school_df = pd.DataFrame(school_data)
school_df.info()

Error: 400
{"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 1000."}}


NameError: name 'location' is not defined

In [ ]:
# Create DF
schools_df = school_df[['Name', 'Address', 'Ratings', 'Latitude', 'Longitude']].copy()

# Rename columns
schools_df.columns = ['school_name', 'school_address', 'school_ratings', 'latitude', 'longitude']

# Add new column for school_id
schools_df['school_id'] = range(1, len(schools_df) + 1)

# Reorder columns
schools_df = schools_df[['school_id', 'school_name', 'school_address', 'school_ratings', 'latitude', 'longitude']]
print(schools_df.head())

   school_id                    school_name  \
0          1  Magellan International School   
1          2    Austin International School   
2          3           McCallum High School   
3          4         Gorzycki Middle School   
4          5  International School of Texas   

                               school_address  school_ratings   latitude  \
0  7938 Great Northern Blvd, Austin, TX 78757             4.5  30.363261   
1           4001 Adelphi Ln, Austin, TX 78727             4.5  30.422421   
2          5600 Sunshine Dr, Austin, TX 78756             3.5  30.326240   
3       7412 W Slaughter Ln, Austin, TX 78749             4.0  30.216398   
4          4402 Hudson Bend, Austin, TX 78734             4.5  30.404580   

   longitude  
0 -97.742185  
1 -97.718074  
2 -97.729820  
3 -97.893745  
4 -97.927680  


In [ ]:
# Export DF as CSV file
schools_df.to_csv("Resources/schools.csv", index=False)

In [ ]:
# Load property data from CSV
property_df = pd.read_csv("Resources/property.csv")

In [ ]:
from geopy.distance import great_circle

# Junction Table for property_school
property_school_df = pd.DataFrame(columns=['property_id', 'school_id'])

# Find nearby locations and create relationships
for property_row in property_df.itertuples():
    property_location = (property_row.latitude, property_row.longitude)
    
    for school_row in schools_df.itertuples():
        school_location = (school_row.latitude, school_row.longitude)
        
        # Calculate distance between property and school locations
        distance_km = great_circle(property_location, school_location).kilometers
        
        distance_threshold = 1.0  
        
        # If the distance is within the threshold, create a relationship
        if distance_km <= distance_threshold:
            new_row = pd.DataFrame({
                'property_id': [property_row.property_id],
                'school_id': [school_row.school_id]
            })
            property_school_df = pd.concat([property_school_df, new_row], ignore_index=True)

# Display the resulting DataFrame
print(property_school_df)


      property_id school_id
0               2       127
1               2       850
2               5       127
3               6       745
4               6       850
...           ...       ...
21962        8879       670
21963        8879       763
21964        8880       492
21965        8880       670
21966        8880       763

[21967 rows x 2 columns]


In [ ]:
# Export junction table as CSV file
property_school_df.to_csv("Resources/property_school.csv", index=False)